In [ ]:
- m pip install pysrt


: 

In [ ]:
pip install gradio

: 

In [ ]:
!pip install SpeechRecognition


: 

In [ ]:
import joblib
import gradio as gr
import numpy as np
import string
import nltk
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
import speech_recognition as sr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import joblib
import pysrt


: 

In [ ]:
tfidf_vectorizer = joblib.load(r'c:\Users\mani chourasiya\Desktop\day_9ToxicCommentClassifierUI\practice-repo-main\tfidf_vectorizer.pkl')
lgbm_model = joblib.load(r'C:\Users\mani chourasiya\Desktop\day_9ToxicCommentClassifierUI\practice-repo-main\lgbm_multi_classifier.pkl')

: 

In [ ]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

: 

In [ ]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()



# Function to preprocess text
def preprocessing(text):
    # Convert text to lowercase
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]

    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into text
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text



: 

In [ ]:
# Function to predict toxicity
def predict_toxicity(text):
    processed_text = preprocessing(text)
    text_tfidf = tfidf_vectorizer.transform([processed_text])
    prediction = lgbm_model.predict(text_tfidf)
    probabilities = lgbm_model.predict_proba(text_tfidf)

    class_names = ['toxic', 'severe toxic', 'obscene', 'threat', 'insult', 'identity hate']

    toxic_classes = []
    toxic_probabilities = []

    for i in range(len(prediction[0])):
        if prediction[0][i] == 1:
            toxic_classes.append(class_names[i])
            toxic_probabilities.append(probabilities[i][0][1])

    if toxic_classes:
        result = "⚠️ This comment has been flagged as toxic. ⚠️\n\nIntensity Breakdown:\n"
        sorted_indices = np.argsort(toxic_probabilities)[::-1]
        for idx in sorted_indices:
            result += f"{toxic_classes[idx]}: {toxic_probabilities[idx]*100:.2f}%\n"
    else:
        result = "✅ This comment is safe and non-toxic. ✅"

    return result



: 

In [ ]:
# Function to convert video to audio
def video_to_audio(video_file):
    video_clip = VideoFileClip(video_file)
    audio_path = "audio.mp3"
    video_clip.audio.write_audiofile(audio_path)
    return audio_path



: 

In [ ]:

# Load the subtitle file
subs = pysrt.open('path_to_your_file.srt')

# Extract captions
captions = []
for sub in subs:
    captions.append(sub.text)

# Combine all captions into a single string
all_captions = ' '.join(captions)
print(all_captions)


: 

In [ ]:
# Function to convert audio to text
def audio_to_text(audio_path):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_mp3(audio_path)
    audio.export("temp.wav", format="wav")

    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_google(audio_data)

    os.remove("temp.wav")
    return text



: 

In [ ]:
# Combined function to handle video input and return toxicity analysis
def analyze_video_toxicity(video_file):
    audio_path = video_to_audio(video_file)
    text = audio_to_text(audio_path)
    toxicity_result = predict_toxicity(text)
    return text, toxicity_result



: 

In [ ]:
# Example text data and labels to fit the tfidf_vectorizer and model (replace with your own dataset)
example_texts = [ "bad", "awful", "terrible", "horrible", "hate", "stupid", "idiot", "dumb", "fool", "ugly",
    "disgusting", "nasty", "gross", "suck", "loser", "moron", "jerk", "creep", "freak",
    "pathetic", "worthless", "trash", "garbage", "scum", "pig", "dog", "filth", "slime",
    "bastard", "asshole", "shit", "crap", "piss", "bitch", "whore", "slut", "dick", "cock",
    "cunt", "fag", "dyke", "retard", "loser", "waste", "failure", "kill", "die", "fuck",
    "good", "great", "awesome", "fantastic", "love", "smart", "intelligent", "brilliant", "genius", "beautiful"
]

example_labels = ["⚠️ This comment has been flagged as toxic. ⚠️\n\nIntensity Breakdown:\n"] * 48 + ["✅ This comment is safe and non-toxic. ✅"] * 10 # Example labels, now with two classes

# Initialize and fit the pipeline model
model = make_pipeline(TfidfVectorizer(), LogisticRegression())
model.fit(example_texts, example_labels)
def analyze_video_toxicity(video_file):
    # Extract audio and convert to text (placeholder function)
    clip = VideoFileClip(video_file)
    audio_text = "Extracted text from the video audio."  # Replace with actual audio-to-text conversion

    # Predict toxicity
    toxicity_result = model.predict([audio_text])

    return audio_text, toxicity_result

def analyze_and_print(video_file):
    converted_text, toxicity_result = analyze_video_toxicity(video_file)
    print(f"Text from the video: {converted_text}")
    return toxicity_result

interface = gr.Interface(
    fn=analyze_and_print,
    inputs=gr.Video(label="Upload your video"),
    outputs=gr.Textbox(label="Toxicity Analysis"),
    title="Video Toxicity Classifier",
    description="Upload a video to analyze its audio content for toxicity.",
    theme="default",
    allow_flagging='never'
)

if __name__ == "__main__":
    interface.launch(debug=True)


: 